**BEFORE YOU COPY AND EDIT NOTEBOOK, PLEASE SUPPORT AND UPVOTE**

v10: Switch to EfficientNet-B5 and larger image size (456x456) with Ranger, gradient accumulation, mixup

v7: Switch to EfficientNet-B4 and larger image size (380x380)

v6: Back to 10 epochs (public LB: 0.884)

v5: Switch to 5 epochs of EfficientNet-B3 model with Ranger (w/ flat+cosine anneal LR), 8x TTA (public LB: 0.881)

v4: Train 5-->10 epochs (public LB: 0.874)

v3: add TTA (public LB: 0.873)

v2: add mixup and label smoothing (public LB: 0.870)

v1: Initial version (public LB: 0.860)


# Cassava Leaf Disease Classification - Simple EDA and fastai starter

In this competition, we are trying to identify common diseases of cassava crops using data science and machine learning.
Previous methods of disease detection require farmers to solicit the help of government-funded agricultural experts to visually inspect and diagnose the plants. This suffers from being labor-intensive, low-supply and costly. Instead, it would be preferred if an automated pipeline based on mobile-quality photos of the cassava leafs could be developed.

This competition provides a farmer-crowdsourced dataset, labeled by experts at the National Crops Resources Research Institute (NaCRRI).

In this kernel, I will present a quick 'n dirty EDA and fastai starter. 


## A look at the data

Let's start out by setting up our environment by installing and importing the required modules and setting a random seed:

In [ ]:
!pip install ../input/pytorch-image-models/timm-0.3.1-py3-none-any.whl

In [ ]:
import numpy as np
import os
import pandas as pd
from fastai.vision.all import *
import albumentations

In [ ]:
set_seed(999,reproducible=True)

Let's check what is available to us:

In [ ]:
dataset_path = Path('../input/cassava-leaf-disease-classification')
os.listdir(dataset_path)

We can see that we have our train csv file with the train image names and labels, the sample submission csv with the test image names, and the train and test image folders. We also have the images in tfrecords format which is useful for quick loading of images, especially for TensorFlow and TPUs. We won't use this for today though.

Let's check the train csv file:

In [ ]:
train_df = pd.read_csv(dataset_path/'train.csv')

In [ ]:
train_df.head()

Let's do some quick processing of the image filenames to make it easier to access:

In [ ]:
train_df['path'] = train_df['image_id'].map(lambda x:dataset_path/'train_images'/x)
train_df = train_df.drop(columns=['image_id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head(10)

Okay let's check how many images are available in the training dataset:

In [ ]:
len_df = len(train_df)
print(f"There are {len_df} images")


We have >21,000 images! Hopefully, we can develop a highly-predictive, robust, and generalizable model with this dataset. 

Let's check the distribution of the different classes:

In [ ]:
train_df['label'].hist(figsize = (10, 5))


In this case, we have 5 labels (4 diseases and healthy):
0. Cassava Bacterial Blight (CBB)
1. Cassava Brown Streak Disease (CBSD)
2. Cassava Green Mottle (CGM)
3. Cassava Mosaic Disease (CMD)
4. Healthy

In this case label 3, [Cassava Mosaic Disease (CMD)](https://en.wikipedia.org/wiki/Cassava_mosaic_virus) is the most common label. This imbalance may have to be addressed with a weighted loss function or oversampling. I might try this in a future iteration of this kernel or in a new kernel.

Let's check an example image to see what it looks like:

In [ ]:
from PIL import Image

im = Image.open(train_df['path'][1])
width, height = im.size
print(width,height) 

In [ ]:
im

## Data loading

After my quick 'n dirty EDA, let's load the data into fastai as `DataLoaders` objects. 

First let's define item and batch transforms. These transforms are the albumentations transforms applied in fastai with the help of some code written by @muellerzr (see [here](https://www.kaggle.com/muellerzr/recreating-abhishek-s-tez-with-fastai) for more details). The batch size is set to 32 here.


In [ ]:
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [ ]:
def get_train_aug(sz): return albumentations.Compose([
            albumentations.RandomResizedCrop(sz,sz),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(sz): return albumentations.Compose([
    albumentations.CenterCrop(sz,sz, p=1.),
    albumentations.Resize(sz,sz)
], p=1.)

While fastai provides various ways of doing custom dataloading (even just taking plain PyTorch DataLoaders), traditional image classification problems work well the high-level data API. Here, we can pass all the required info to create an `DataLoaders` object.

In [ ]:
def get_dls(sz,bs):
    item_tfms = AlbumentationsTransform(get_train_aug(sz), get_valid_aug(sz))
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]
    dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                   valid_pct=0.2, #80-20 train-validation random split
                                   seed=999, #seed
                                   label_col=0, #label is in the first column of the DataFrame
                                   fn_col=1, #filename/path is in the second column of the DataFrame
                                   bs=bs, #pass in batch size
                                   item_tfms=item_tfms, #pass in item_tfms
                                   batch_tfms=batch_tfms) #pass in batch_tfms
    return dls

In [ ]:
dls = get_dls(456,16)

To confirm successful dataloader creation, we can use the `show_batch` command, which shows a subset of the batch:

In [ ]:
dls.show_batch()

## Model training:

Let's train a simple EfficientNet-B5 model. We will use the wonderful [timm](https://github.com/rwightman/pytorch-image-models) package by Ross Wightman to define the model. Since this competition doesn't allow internet access, I have added the pretrained weights from timm as a dataset, and the below code cell will allow timm to find the file:

In [ ]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/timmefficientnet/tf_efficientnet_b5_ns-6f26d0cf.pth' '/root/.cache/torch/hub/checkpoints/tf_efficientnet_b5_ns-6f26d0cf.pth'

In fastai, the trainer class is the `Learner`, which takes in the data, model, optimizer, loss function, etc. and allows you to train models, make predictions, etc.

When training common CNN models like ResNets, we typically can use the `cnn_learner` function which creates a `Learner` object that allows us to train a provided model with the given dataloaders. However, `cnn_learner` doesn't support the models from timm out-of-the-box. Zach Mueller (@muellerzr) [has written some simple functions](https://walkwithfastai.com/vision.external.timm) to make it very easy to create Learner objects for timm models.

In [ ]:
from timm import create_model
from fastai.vision.learner import _update_first_layer

def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")
        
def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children())) * (2 if concat_pool else 1)
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

In [ ]:
def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

Let's now create our `Learner` object. We will also use common state-of-the-art training techniques like label smoothing and Ranger optimizer, which are provided in fastai. We can also use mixed precision very easily:

In [ ]:
learn = timm_learner(dls, 
                    'tf_efficientnet_b5_ns',
                     opt_func=ranger,
                     loss_func=LabelSmoothingCrossEntropy(),
                     cbs=[GradientAccumulation(n_acc=32)],
                     metrics = [accuracy]).to_native_fp16()

We are now provided with a Learner object which has a frozen model (only the weights of the head of the model can be updated). In order to train a model, we need to find the most optimal learning rate, which can be done with fastai's learning rate finder:

In [ ]:
learn.lr_find()

Let's start training the model. Often, the best way to train a model is to train the frozen pretrained model for a single epoch then train the whole pretrained model for several epochs. The Ranger optimizer performs best with a flat+cosine annealing learning rate schedule. We will now train the frozen model for one epoch.

As shown above, the optimal learning rate for training the frozen model is where the loss is decreasing most rapidly: around ~1e-1. To be safe, I will use high weight decay to help prevent overfitting. We will also use another common state-of-the-art training technique: mixup.

In [ ]:
learn.freeze()
learn.fit_flat_cos(1,1e-1, wd=0.1, cbs=[MixUp()])

In [ ]:
learn.save('stage-1')

In [ ]:
learn = learn.load('stage-1')

In [ ]:
learn.recorder.plot_loss()

Let's now unfreeze the model and find a good learning rate:

In [ ]:
learn.unfreeze()
learn.lr_find()

Let's train for 10 epochs with the unfrozen model.

In [ ]:
learn.unfreeze()
learn.fit_flat_cos(10, 1e-3,cbs=[MixUp(),SaveModelCallback()])

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn = learn.to_native_fp32()

In [ ]:
learn.save('stage-2')

We plotted the loss, put the model back to fp32, and now we can export the model if we want to use later (i.e. for an inference kernel):

In [ ]:
learn.export()

## Inference

It's very simple to perform inference with fastai. The `dls.test_dl` function allows you to create test dataloader using the same pipeline defined earlier.

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df.head()

In [ ]:
_sample_df = sample_df.copy()
_sample_df['path'] = _sample_df['image_id'].map(lambda x:dataset_path/'test_images'/x)
_sample_df = _sample_df.drop(columns=['image_id'])
test_dl = dls.test_dl(_sample_df)

We can easily confirm that the test_dl is correct:

In [ ]:
test_dl.show_batch()

Now let's pass the dataloader to the model and get predictions. We will use a common inference technique known as test-time augmentation (average predictions when passing in various augmented versions of the test image). This is also implemented in fastai. Let's do 8x TTA:

In [ ]:
preds, _ = learn.tta(dl=test_dl, n=15, beta=0)

Let's make a submission with these predictions!

In [ ]:
sample_df['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sample_df.to_csv('submission.csv',index=False)

Now, **WE ARE DONE**!

If you enjoyed this kernel, please give it an upvote. If you have any questions or suggestions, please leave a comment!